# Reference model constraint

In [1]:
# how many n soundings
# create matix with n columns
# use standard/input parameters
# allow user to input own parameters at certain location

## cell by sounding n or by coordinates??

In [2]:
#import libraries
import pandas as pd
import numpy as np
import xarray as xr

In [3]:
## User requirement: enter the folder location where the .dat file is saved
folder= str(r"C:/Users/u67397/AnacondaProjects/aem/input_data/")

## User requirement: enter the name of the second .dat file (without the ".dat" text)
dat_name= str("AA140005_Line1080")

## User requirement: enter the reference model conductivity specified for result file
reference_EC1= 0.001

## User requirement: enter the number of layers result file
layers= 30

## User requirement: enter the name of the second .dat file (without the ".dat" text)
con_name= str("galeisbstdem")

In [4]:
## User requirement: enter the folder location where to save files into

output_folder= str(r"C:/Users/u67397/AnacondaProjects/aem/output_data")

In [5]:
#Enter additional information. Create tests. e.g. 3 layers at xxx location with xxx EC values

In [6]:
dat= pd.read_fwf(folder+dat_name+".dat", header=None)
datlen= len(dat.index)

In [7]:
#thickness from con file
con = pd.read_csv (folder+con_name+".con", header=None) #import control file
con.replace(regex=True,to_replace=r'\t', value=r'', inplace=True)#remove tabs
tlayers=con.loc[con[0].str.contains('Thickness      = ')] #create a dataframe that contains layer thickness saved in confile (.con)
thicknessoflayers= tlayers.replace(regex=True,to_replace=r'Thickness      = ', value=r'')
thicknessoflayers=thicknessoflayers[0].str.split((' '),expand=True).replace('',np.nan)
thicknessoflayers.dropna(axis=1,inplace=True)
thicknessoflayers=thicknessoflayers.transpose().astype(float)
thicknessoflayers=np.cumsum(thicknessoflayers)
thicknessoflayers=thicknessoflayers.transpose()
thicknessoflayers=(pd.np.tile(thicknessoflayers, (datlen, 1)))
thicknessoflayers=pd.DataFrame(thicknessoflayers)

In [8]:
EC=pd.Series(reference_EC1)
EC=(pd.np.tile(EC, (datlen, layers)))
EC=pd.DataFrame(EC)

In [25]:
#join datafile and layer thickness dataframes
alldata=pd.concat([dat,thicknessoflayers,EC], axis=1, join='outer',ignore_index=True)

In [24]:
##Weighting - adjust to hit reference model. set out to side and botom x layers. 
## input constraint to relax in horiz space

In [11]:
alldata=alldata.set_index([2,3], drop=False)
alldata['all_sumcoord']=alldata[2]+alldata[3]

In [12]:
#open constraint csv
constraint_name=str("input_test") 
constraint_data= pd.read_csv(folder+constraint_name+".csv") #import control file
constraint_data['const_sumcoord']=constraint_data['x']+constraint_data['y']
constraint_data=constraint_data.set_index(['ID'], drop=False)

In [23]:
matched=pd.DataFrame([0,0,0]).transpose().set_index([0])

for x in range(len(constraint_data)):
    g=(constraint_data.ID[x])
    h=(constraint_data.const_sumcoord[x])
    alldata[str(g)]= np.abs(alldata.all_sumcoord-h) 
    matched1=np.argmin(alldata[str(g)])
    e_matched=matched1[0]
    n_matched=matched1[1]
    matched2=pd.DataFrame([g,float(e_matched),float(n_matched)]).transpose().set_index([0]).astype(float)
    matched=pd.concat([matched,matched2], axis=0)

matched=matched.rename(columns={1:'E_matched',2:'N_matched'}).drop([0])

In [22]:
constraint_data2= pd.concat([constraint_data, matched],axis=1)
constraint_data2=constraint_data2.set_index(['E_matched','N_matched'])

In [20]:
#create subset of three dataframes and reformat

#EC
EC_1=EC.iloc[0:5,0:5]

#thickness
thickness_1=thicknessoflayers.iloc[0:5,0:5]
thickness_2= thickness_1[0:1]
thickness_2=np.array(thickness_2).squeeze()

#coordinates
coords_1=pd.DataFrame(dat.iloc[0:5,2:3])
coords_2=coords_1.transpose()
coords_2=np.array(coords_2).squeeze()

In [17]:
#Convert datasets into arrays

#EC array
coords1={'location': coords_2, 'depth':thickness_2}
ec_da = xr.DataArray(EC_1, dims=('location', 'depth'), coords=coords1)

coords2={'location': coords_2, 'layer': [1, 2, 3, 4, 5]}
thickness_da = xr.DataArray(thickness_1, dims=('location', 'layer'), coords=coords2)

dataset = xr.Dataset({'ec': ec_da, 'thickness_name': thickness_da})

In [26]:
array2=dataset.assign(ec=0.05).where(thickness_da <10).sel(location=220926.1)
array2['ec']=array2.ec.fillna(0.001)
array3=np.array(array2['ec'])

In [27]:
ec_da.loc[220926.1]=array3
dataset_da = xr.Dataset({'ec': ec_da, 'thickness_name': thickness_da})
dataset_da.ec

<xarray.DataArray 'ec' (location: 5, depth: 5)>
array([[ 0.05 ,  0.05 ,  0.05 ,  0.001,  0.001],
       [ 0.001,  0.001,  0.001,  0.001,  0.001],
       [ 0.001,  0.001,  0.001,  0.001,  0.001],
       [ 0.001,  0.001,  0.001,  0.001,  0.001],
       [ 0.001,  0.001,  0.001,  0.001,  0.001]])
Coordinates:
  * location  (location) float64 2.209e+05 2.209e+05 2.209e+05 2.209e+05 ...
  * depth     (depth) float64 3.0 6.3 9.92 13.92 18.32

In [9]:
# coords_m=dat.iloc[:,1:3]
# EC_1= xr.Dataset(EC.iloc[0:9,0:9])
# thickness_m=xr.Dataset(thicknessoflayers.iloc[0:9,0:9])
# dat_m=xr.Dataset(dat.iloc[0:9,0:9])


In [ ]:
# dataset = xarray.Dataset({'ec': ec_da, 'thickness_name': thickness_da})
# dataset.ec

In [ ]:
# coords={'location': coords_test, 'layer': [1, 2, 3, 4, 5]}
# thickness_da = xarray.DataArray(thickness_m, dims=('location', 'layer'), coords=coords)

In [ ]:
# thickness_test= thickness_m[0:1]
# thickness_test=np.array(thickness_test).squeeze()
# thickness_test

In [ ]:
# coords_test= coords_m.transpose()
# coords_test=np.array(coords_test).squeeze()

In [ ]:
# dataset.sel(location=220926.1).update({dataset.ec:('ec2',[0.05,  0.05,  0.01,  0.01,  0.01])}, inplace=True)
# dataset.sel(location=220926.1).update({'ec':('ec2',[0.05,  0.05,  0.01,  0.01,  0.01])}, inplace=True)
#dataset.location[0]#=dataset.sel(location=220926.1)#.update({dataset.ec:('ec2',[0.05,  0.05,  0.01,  0.01,  0.01])}, inplace=True)
# dataset.loc['location',['220926.1']]

#dataset.update({'ec':('ec',[1,2,3])})
#dataset.update({(dataset.sel(location=220926.1).ec):(1)})
# dataset.ec

In [ ]:
#create and save a .csv that contains depth of investigation data for input into Discover PA software
alldata.to_csv(output_folder+'/depth_data.csv') #save doi data as csv

In [ ]:
# test2= dataset.sel(location=220926.1)
# test2=test2.thickness_name<10#.ec
# test2

In [ ]:
# dataset.ec.isel_points(location=(dataset.thickness_name < 10).location)

In [ ]:
# dataset.where(dataset.sel(location=51).thickness_name < 10).fillna(100)

In [ ]:
# dataset.location[:] = [5, 6, 7, 8, 9]

In [ ]:

#coords_m=pd.DataFrame(dat.iloc[0:5,2:4])

# test=pd.concat([coords_m,thickness_m,EC_m],axis=1,ignore_index=True).set_index([0,1])
# test=pd.concat([test,constraint_data2],axis=1)
# test 


# thickness_m=pd.concat([coords_m,thickness_m],axis=1,ignore_index=True)#.set_index([0,1])
# array=np.array([[coords_m],[thickness_m],[EC_m]])
# array.shape

In [ ]:
# for row in constraint_data:
#     print(row)
    #sample[str(row)]= 0

#print(sample)
    
    
#     print('loop1_start')
#     print(row)
#     print('loop1_ends')
#     for blah in constraint_data['const_sumcoord']:
#         print('loop2_starts')
#         print (blah)
#         print('loop2_ends')
# #         while x>0:
#         for rah in sample:
#             sample[str(row)]=np.abs(test.all_sumcoord-blah)
# #                 x=x-1
#             print(sample)
                

In [ ]:
## find coordinate
## enter layer thickness (e.g 3)
## extrapolate between the points to maximum layers (e.g. 30)

In [ ]:
# array=np.array([[dat],[thicknessoflayers],[EC]],ndmin=3)
# array.shape
# #test=np.ndarray(array)
# array[[0,0,0],[[0,0,0],[0,0,0]]]

In [ ]:
# #create a dataframe that contains layer thickness saved in confile (.con)
# tlayers=con.loc[con[0].str.contains('Thickness      = ')]
# thicknessoflayers= tlayers.replace(regex=True,to_replace=r'Thickness      = ', value=r'')
# thicknessoflayers=thicknessoflayers[0].str.split((' '),expand=True).replace('',np.nan)
# thicknessoflayers.dropna(axis=1,inplace=True)
# thicknessoflayers=(pd.np.tile(thicknessoflayers, (datlen, 1)))
# #thicknessoflayers=pd.DataFrame(thicknessoflayers)
# thicknessoflayers

In [ ]:
# thicknessoflayers=(pd.np.tile(thicknessoflayers, (datlen, 1)))


In [ ]:
def coord(easting,northing):
    easting=int(easting)
    northing=int(northing)
    print(type(easting))

coord('1','2')

In [ ]:
def print_text():
    print("complete")

In [ ]:
a = [[1, 2, 3, 4], [5, 6], [7, 8, 9]]
for i in range(len(a)):
    for j in range(len(a[i])):
        print(a[i][j], end=' ')
    print()

In [ ]:
# EC_x=xr.Dataset([EC_m],[thickness_m])
# EC_x